In [ ]:
# This notebooks looks at the timestamps of the atmcs to make sure
# everything is consistent

In [1]:
import asyncio 
import matplotlib

import numpy as np
import pandas as pd

from matplotlib import pylab as plt
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient, rendezvous_dataframes

In [ ]:
%matplotlib inline

In [ ]:
efd_client = EfdClient('summit_efd')
#efd_client = EfdClient('ldf_stable_efd') 

Query for all the `endReadout` events on the timespan of the night, elevation, and pmd data

In [ ]:
date='20210817'
test='time_test'
run='initial'

if run == 'initial': 
# Starting Test1, Nominal LUT from m1_hex_20210810_v3.yaml:v0.8.2.alpha.2-0-g72c6e32
# start time is 2021-08-12T21:01:52.999
    t1 = Time("2021-08-17T17:36:45", format='isot', scale='tai')
    t2 = Time("2021-08-17T17:39:20.386", format='isot', scale='tai')
    t2 = Time("2021-08-17T17:42:00.386", format='isot', scale='tai') # half
#     t2 = Time("2021-08-17T17:45:00.386", format='isot', scale='tai') # original
    scale_factor=1.0
    start_log_msg = '[2021-07-08_Repeat_Focus_Test_START]'
    finish_log_msg = '[2021-07-08_Repeat_Focus_Test_END]'
# elif run == 'test2':
# Starting Test2, LUT offset by 1211.6539141845715 and is now 122385.77740478516
# start time is 2021-08-12T21:21:00.763
#     t1 = Time("2021-08-12T21:21:00.763", format='isot', scale='tai')
#     t2 = Time("2021-08-12T21:28:08.745", format='isot', scale='tai')
#     scale_factor=1.01
else:
    raise IOError('Not a valid input')

In [ ]:
pmd = await efd_client.select_time_series("lsst.sal.PMD.position", ["position0", "position1", "position2", "position3", "position4", "position5", "private_efdStamp"], t1, t2)

In [ ]:
pmd

In [ ]:
atmcs_target = await efd_client.select_time_series("lsst.sal.ATMCS.command_trackTarget", 
                                           ["private_efdStamp","elevation"], t1, t2)
atmcs_target

In [ ]:
atptg = await efd_client.select_time_series("lsst.sal.ATPtg.command_azElTarget", 
                                           ["private_efdStamp","azDegs","elDegs",], t1, t2)

In [ ]:
atptg

In [ ]:
el0 = await efd_client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", ["elevationCalculatedAngle0", "private_efdStamp", "cRIO_timestamp", "private_sndStamp"], t1, t2)

In [ ]:
el=el0

In [ ]:
# Check that the private_efdStamp is used as the index in the EFD

In [ ]:
el_time=Time(el.private_efdStamp,format='unix', scale='tai')

In [ ]:
# This should be < a milli-second or so
np.timedelta64(el.index.values[0]-el_time.datetime64[0],'us')

In [ ]:
# Check a salobj CSC just for a consistency check
pmd_time=Time(pmd.private_efdStamp,format='unix', scale='tai')
np.timedelta64(pmd.index.values[0]-pmd_time.datetime64[0],'us')

In [ ]:
# Check that the cRIO_timestamp is close to the private_sndStamp, which are both TAI but known to differ by 2-3 seconds

In [ ]:
el_cRIO_time=Time(el.cRIO_timestamp,format='unix', scale='tai')
np.timedelta64(el.index.values[0]-el_cRIO_time.datetime64[0],'s')

In [ ]:
# Check that the cRIO_timestamp is ~37s from private_efdStamp (which is UTC)

In [ ]:
el_time_TAI=Time(el.private_sndStamp,format='unix', scale='tai')
np.timedelta64(el_time_TAI.datetime64[0]-el_cRIO_time.datetime64[0],'s')

In [ ]:
pmd.index.values[0]-pmd_time.datetime64[0]

In [ ]:
tmp2=pd.to_datetime(el.index.values).to_series()

In [ ]:
el_time.datetime64-tmp2

In [ ]:
# first verify that private_sndStamp is being used as the index!

In [ ]:
atmcs_inPos = await efd_client.select_time_series("lsst.sal.ATMCS.logevent_allAxesInPosition", ["inPosition", "private_efdStamp"], t1, t2)
# atmcs_inPos

In [ ]:
pressure = await efd_client.select_time_series("lsst.sal.ATPneumatics.m1AirPressure", ["pressure", "private_efdStamp"], t1, t2)

In [ ]:
fig_height=10
fig_width=20
nwide=1; nhigh=1
el=el0
fig, (row1) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
fig.suptitle('Using Select_time_series with no offset')
xvals = (el.private_efdStamp) # um
yvals = el.elevationCalculatedAngle0
row1.plot(xvals, yvals, 'o-',label='current elevation')
row1.set_ylabel('Elevation [deg]')
row1.set_xlabel('Time [s]')
# overplot commanded position
xvals = (atmcs_target.private_efdStamp) # um
yvals = atmcs_target.elevation
row1.plot(xvals, yvals, '.',color='black', label='Commanded Position',markersize=1)
#overplot command of new position
xvals = atptg.private_efdStamp
yvals = atptg.elDegs
purple_label = 'New_target_command'
for i,val in enumerate(yvals):
    row1.vlines(xvals.iloc[i],ymin=0,ymax=90, color='purple', label=purple_label)
    purple_label=None
#overplot the in-position
xvals = atmcs_inPos.private_efdStamp
yvals = atmcs_inPos.inPosition
grn_label = 'Mount In-Position'
red_label = 'Mount Out-of-Position'
for i,val in enumerate(yvals):
    if val:
        # In position
        row1.vlines(xvals.iloc[i],ymin=0,ymax=90, color='green', label=grn_label)
        grn_label=None
        continue
    else:
        row1.vlines(xvals.iloc[i],ymin=0,ymax=90, color='red', label=red_label)
        red_label=None

row1.legend()

In [ ]:
# el1 = await efd_client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", ["elevationCalculatedAngle0", "private_efdStamp", "cRIO_timestamp", "private_sndStamp"], t1, t2) 
el1 = await efd_client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", ["elevationCalculatedAngle"], t1, t2)
# cRIO timestamp report in TAI
el1.times=el1.times-37

In [ ]:
el1

In [ ]:
el1.keys()

In [ ]:
el_index_time=Time(el1.times,format='unix', scale='tai')

In [ ]:
nwide=1; nhigh=1
fig, (row1) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
fig.suptitle('Using select_packed_time_series with no offset')
el=el1
xvals = (el.times) # um
xvals = el_index_time.unix # UTC
yvals = el.elevationCalculatedAngle
row1.plot(xvals, yvals, 'o-')
row1.set_ylabel('Elevation [deg]')
row1.set_xlabel('Time [s]')
# overplot commanded position
xvals = (atmcs_target.private_efdStamp) # um
yvals = atmcs_target.elevation
row1.plot(xvals, yvals, 'x',color='black', label='Commanded Position',markersize=1)
#overplot command of new position
xvals = atptg.private_efdStamp
yvals = atptg.elDegs
purple_label = 'New_target_command'
for i,val in enumerate(yvals):
    row1.vlines(xvals.iloc[i],ymin=0,ymax=90, color='purple', label=purple_label)
    purple_label=None
#overplot the in-position
xvals = atmcs_inPos.private_efdStamp
yvals = atmcs_inPos.inPosition
grn_label = 'Mount In-Position'
red_label = 'Mount Out-of-Position'
for i,val in enumerate(yvals):
    if val:
        # In position
        row1.vlines(xvals.iloc[i],ymin=0,ymax=90, color='green', label=grn_label)
        grn_label=None
        continue
    else:
        row1.vlines(xvals.iloc[i],ymin=0,ymax=90, color='red', label=red_label)
        red_label=None

row1.legend()

In [2]:
efd_client = EfdClient('summit_efd')

In [3]:
t1 = Time("2021-08-17T17:36:45", format='isot', scale='tai')
t2 = Time("2021-08-17T17:42:00.386", format='isot', scale='tai') # half

In [4]:
pmd = await efd_client.select_time_series("lsst.sal.PMD.position", ["position0", "private_efdStamp", "private_sndStamp"], t1, t2)

In [5]:
# Check that the private_efdStamp is used as the index in the EFD

In [6]:
pmd_time=Time(pmd.private_efdStamp,format='unix', scale='tai')

In [7]:
# This should be < a milli-second or so
np.timedelta64(pmd.index.values[0]-pmd_time.datetime64[0],'us')

numpy.timedelta64(-300,'us')

In [9]:
# Compare private_sndStamp to the index
pmd_sndStamp=Time(pmd.private_sndStamp,format='unix', scale='tai')
np.timedelta64(pmd.index.values[0]-pmd_sndStamp.datetime64[0],'us')

numpy.timedelta64(-37000300,'us')

In [13]:
np.timedelta64(pmd.index.values[0]-pmd_sndStamp.datetime64[0],'us')

numpy.timedelta64(-37000300,'us')

In [14]:
np.timedelta64(pmd.index.values[0]-pmd_sndStamp.datetime64[0],'s')

numpy.timedelta64(-38,'s')

In [ ]:
import astropy as astropy

In [20]:
astropy.__version__

'4.1'